In [1]:
# Importing pandas library
import pandas as pd
import numpy as np
import random

In [2]:
# Importing the csv data files 
metadata = pd.read_csv('metadata_integrated.csv',error_bad_lines=False, warn_bad_lines=True)
rating_data = pd.read_csv('ratings_integrated.csv',error_bad_lines=False, warn_bad_lines=True)

# For S3.8 [optional]: Applying S3.2-S3.7 under another pair of Training and Test sets 

# Change the value of "test_sample_size" to 10

In [3]:
# Setting the test sample size
test_sample_size = 5

In [4]:
unique_user_ID = rating_data['userId'].unique()
print(unique_user_ID)
print(len(unique_user_ID))

unique_movie_ID = metadata['movieId'].unique()
print(unique_movie_ID)
print(len(unique_movie_ID))

[  1   7  31  32  36  39  73  88  96 110 111 150 161 165 186 242 254 288
 310 311 325 338 341 386 394 452 457 461 468 485 487 496 511 516 525 590
 596 601 607 619 639 641  19  35  61  99 102 128 175 185 188 213 214 268
 294 303 363 387 420 442 466 471 472 514 518 529 530 547 551 564 575 577
 582 602 603 605 646 665  23  30  70  90 105 119 141 212 233 252 292 306
 318 380 507 509 534 537 584 608 647  21  34  41  49  57  66  77 103 118
 155 177 195 248 276 299 346 351 353 358 390 423 463 522 580 585 597 624
 634 654  38  56  94 130 133 148 198 229 232 262 280 283 320 321 330 372
 373 391 430 441 460 481 497 505 510 521 539 545 587  15  17  22 152 257
 265 309 344 370 434 527 533 592 648 661 137 207 313 326 335 388 404 405
 470 488 559 598  20  48  78 211 236 281 419 532 576  81  95 243 247 285
 290 297 300 308 312 337 362 403 560 562 614 617 618 658 664  27 120 291
 342 407 613   4 202 217 439 458 477 546 574  25  26  87 154 224 361 384
 428 502 586 606 220 360 431 436 520 594 669  59 19

In [5]:
rating_data['movieId'].nunique()

9066

In [6]:
metadata['movieId'].nunique()

9042

### Creating the Cartesian Product for all possible combination of UserId and MovieId

In [7]:
# df containing only the userID column
user_df = pd.DataFrame(unique_user_ID, columns = ['userId'])
# Df containing only the movieID column
movies_df = pd.DataFrame(unique_movie_ID, columns = ['movieId'])
print(user_df.head())
print(user_df.size)

print(movies_df.head())
print(movies_df.size)

# Creating the Cartecian Product of Users and Movies
user_movie_product = (
    user_df.assign(key=1)
    .merge(movies_df.assign(key=1), on="key")
    .drop("key", axis=1)
)
print(user_movie_product.head())
print(user_movie_product.info())

   userId
0       1
1       7
2      31
3      32
4      36
671
   movieId
0        1
1        2
2        3
3        4
4        5
9042
   userId  movieId
0       1        1
1       1        2
2       1        3
3       1        4
4       1        5
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6067182 entries, 0 to 6067181
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   userId   int64
 1   movieId  int64
dtypes: int64(2)
memory usage: 138.9 MB
None


In [8]:
# Query for Cross Checking
user_movie_product.loc[(user_movie_product['userId'] == 1) & (user_movie_product['movieId'] == 1029)]

,userId,movieId
830,1,1029


In [9]:
user_movie_product.loc[(user_movie_product['userId'] == 96)  & (user_movie_product['movieId'] == 7587)]

,userId,movieId
77518,96,7587


In [10]:
user_movie_product.head(4)

,userId,movieId
0,1,1
1,1,2
2,1,3
3,1,4


In [11]:
rating_data.head(4)

,userId,movieId,rating,imdbId,tmdbId
0,1,31,2.5,112792,9909.0
1,7,31,3.0,112792,9909.0
2,31,31,4.0,112792,9909.0
3,32,31,4.0,112792,9909.0


In [12]:
rating_data.loc[(rating_data['userId'] == 1) & (rating_data['movieId'] == 1029)]

,userId,movieId,rating,imdbId,tmdbId
42,1,1029,3.0,33563,11360.0


### Merging the dataframes to get the rated and unrated movies for each user

In [13]:
# Merging two dataframes to get the rated and unrated movies
merged_rating_df = pd.merge(user_movie_product, rating_data, how='left', left_on=['userId','movieId'], right_on = ['userId','movieId']).drop(columns = ['imdbId','tmdbId'])

In [14]:
merged_rating_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6067182 entries, 0 to 6067181
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
dtypes: float64(1), int64(2)
memory usage: 185.2 MB


In [15]:
# Cross Checking
merged_rating_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6067182 entries, 0 to 6067181
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
dtypes: float64(1), int64(2)
memory usage: 185.2 MB


In [16]:
merged_rating_df.loc[(merged_rating_df['userId'] == 1) & (merged_rating_df['movieId'] == 1029)]

,userId,movieId,rating
830,1,1029,3.0


In [17]:
merged_rating_df.loc[(merged_rating_df['userId'] == 227) & (merged_rating_df['movieId'] == 162542)]

,userId,movieId,rating
6067178,227,162542,NaN


## Splitting rated and unrated movies for each user

In [18]:
# Dataframe containg only the rated movies by the users
rated_movies = merged_rating_df
rated_movies = rated_movies.dropna()

print(rated_movies.tail())
print(rated_movies.size)
print(rated_movies.info())


bool_series = pd.isnull(merged_rating_df['rating'])
#print(bool_series)  

# Dataframe containg only the un-rated movies by the users
unrated_movies = merged_rating_df[bool_series] 

print(unrated_movies.tail())
print(unrated_movies.size)
print(unrated_movies.info())

         userId  movieId  rating
6060031     227     2385     2.0
6060032     227     2386     5.0
6060033     227     2387     5.0
6060035     227     2389     5.0
6060038     227     2392     4.0
298977
<class 'pandas.core.frame.DataFrame'>
Int64Index: 99659 entries, 30 to 6060038
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   99659 non-null  int64  
 1   movieId  99659 non-null  int64  
 2   rating   99659 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 3.0 MB
None
         userId  movieId  rating
6067177     227   161944     NaN
6067178     227   162542     NaN
6067179     227   162672     NaN
6067180     227   163056     NaN
6067181     227   163949     NaN
17902569
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5967523 entries, 0 to 6067181
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
dtypes: fl

In [19]:
rated_movies.loc[(rated_movies['userId'] == 96) & (rated_movies['movieId'] == 7587)]

,userId,movieId,rating
77518,96,7587,4.0


## Creating test dataset for each user

In [20]:
# Initializing dataframe
column_names = ['userId','movieId','rating']
test_df = pd.DataFrame(columns = column_names)
user_test_df = pd.DataFrame(columns = column_names)

# The follwing selects '5' rated movies for each user for the test dataset
for i in range(len(unique_user_ID)):
    temp = rated_movies.loc[rated_movies['userId'] == unique_user_ID[i]]
    temp_movies = temp['movieId'].tolist()
    sample_movies = random.sample(temp_movies, test_sample_size)
    #print("Random item from list is: ", sample_movies)
    for j in range(len(sample_movies)):
        user_test_df = rated_movies.loc[(rated_movies['userId'] == unique_user_ID[i]) & (rated_movies['movieId'] == sample_movies[j])]
        test_df = pd.concat([test_df, user_test_df])

In [21]:
test_df.head(5)

,userId,movieId,rating
903,1,1129,2.0
1704,1,2150,3.0
927,1,1172,4.0
2920,1,3671,3.0
1661,1,2105,4.0


In [22]:
test_df['userId'] = test_df['userId'].astype(str).astype(int)
test_df['movieId'] = test_df['movieId'].astype(str).astype(int)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3355 entries, 903 to 6059969
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   3355 non-null   int32  
 1   movieId  3355 non-null   int32  
 2   rating   3355 non-null   float64
dtypes: float64(1), int32(2)
memory usage: 78.6 KB


In [23]:
rated_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99659 entries, 30 to 6060038
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   99659 non-null  int64  
 1   movieId  99659 non-null  int64  
 2   rating   99659 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 3.0 MB


## Creating test dataset for each user

In [24]:
# Creating the Train dataset

train_df = pd.merge(rated_movies, test_df, how='left', left_on=['userId','movieId'], right_on = ['userId','movieId'])

bool_series = pd.isnull(train_df['rating_y'])

train_df = train_df[bool_series] 
train_df = train_df.drop(['rating_y'], axis = 1)
train_df = train_df.rename(columns={'rating_x': 'rating'})

In [25]:
train_df.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
5,1,1263,2.0
6,1,1287,2.0


In [26]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96304 entries, 0 to 99658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   96304 non-null  int64  
 1   movieId  96304 non-null  int64  
 2   rating   96304 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.9 MB


In [27]:
# Integrating the features of movie metadata to the train and test dataset
train_df = pd.merge(train_df, metadata, how='left', left_on=['movieId'], right_on = ['movieId'])
test_df = pd.merge(test_df, metadata, how='left', left_on=['movieId'], right_on = ['movieId'])
train_df.head()

,userId,movieId,rating,collectionId,collectionName,genres,imdb_id,original_language,overview,popularity,...,runtime,spoken_languages,status,tagline,title,tmdbId,video,vote_average,vote_count,year
0,1,31,2.5,-99,unknown,"Drama,Crime",112792,en,Former Marine Louanne Johnson lands a gig teac...,9.481338,...,99.0,English,Released,She broke the rules... and changed their lives.,Dangerous Minds,9909,False,6.4,249.0,1995
1,1,1029,3.0,-99,unknown,"Animation,Family",33563,en,Dumbo is a baby elephant born with oversized e...,14.655879,...,64.0,English,Released,The One...The Only...The FABULOUS...,Dumbo,11360,False,6.8,1206.0,1941
2,1,1061,3.0,-99,unknown,"Crime,Drama,Thriller",117665,en,Two gangsters seek revenge on the state jail w...,9.198283,...,147.0,English,Released,When friendship runs deeper than blood.,Sleepers,819,False,7.3,729.0,1996
3,1,1263,2.0,-99,unknown,"Drama,War",77416,en,A group of working-class friends decides to en...,7.729445,...,183.0,"English,Français,Pусский,Tiếng Việt",Released,One of the most important and powerful films o...,The Deer Hunter,11778,False,7.8,943.0,1978
4,1,1287,2.0,-99,unknown,"Action,Adventure,Drama,History",52618,en,Ben-Hur is a 1959 epic film directed by Willia...,14.319743,...,212.0,English,Released,The entertainment experience of a lifetime.,Ben-Hur,665,False,7.5,660.0,1959


In [28]:
test_df.head()

,userId,movieId,rating,collectionId,collectionName,genres,imdb_id,original_language,overview,popularity,...,runtime,spoken_languages,status,tagline,title,tmdbId,video,vote_average,vote_count,year
0,1,1129,2.0,115838,'Escape From ... Collection',"Science Fiction,Action",82340,en,"In 1997, the island of Manhattan has been wall...",10.124544,...,99.0,English,Released,1997. New York City is now a maximum security ...,Escape from New York,1103,False,6.9,720.0,1981
1,1,2150,3.0,87805,'The Gods Must Be Crazy Collection',"Action,Comedy",80801,en,Misery is brought to a small group of Sho in t...,10.973482,...,109.0,"Afrikaans,English",Released,The critics are raving... the natives are rest...,The Gods Must Be Crazy,8393,False,7.1,251.0,1980
2,1,1172,4.0,-99,unknown,"Drama,Romance",95765,it,"A filmmaker recalls his childhood, when he fel...",14.177005,...,124.0,Italiano,Released,"A celebration of youth, friendship, and the ev...",Cinema Paradiso,11216,False,8.2,834.0,1988
3,1,3671,3.0,-99,unknown,"Western,Comedy",71230,en,A town – where everyone seems to be named John...,13.233262,...,93.0,"English,Deutsch,",Released,Never give a saga an even break!,Blazing Saddles,11072,False,7.2,619.0,1974
4,1,2105,4.0,63043,'TRON Collection',"Science Fiction,Action,Adventure",84827,en,As Kevin Flynn searches for proof that he inve...,16.574021,...,96.0,English,Released,A world inside the computer where man has neve...,Tron,97,False,6.6,717.0,1982


In [29]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96304 entries, 0 to 96303
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   userId                96304 non-null  int64  
 1   movieId               96304 non-null  int64  
 2   rating                96304 non-null  float64
 3   collectionId          96304 non-null  int64  
 4   collectionName        96304 non-null  object 
 5   genres                96304 non-null  object 
 6   imdb_id               96304 non-null  int64  
 7   original_language     96304 non-null  object 
 8   overview              96304 non-null  object 
 9   popularity            96304 non-null  float64
 10  production_companies  96304 non-null  object 
 11  production_countries  96304 non-null  object 
 12  runtime               96304 non-null  float64
 13  spoken_languages      96304 non-null  object 
 14  status                96304 non-null  object 
 15  tagline            

In [30]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3355 entries, 0 to 3354
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   userId                3355 non-null   int32  
 1   movieId               3355 non-null   int32  
 2   rating                3355 non-null   float64
 3   collectionId          3355 non-null   int64  
 4   collectionName        3355 non-null   object 
 5   genres                3355 non-null   object 
 6   imdb_id               3355 non-null   int64  
 7   original_language     3355 non-null   object 
 8   overview              3355 non-null   object 
 9   popularity            3355 non-null   float64
 10  production_companies  3355 non-null   object 
 11  production_countries  3355 non-null   object 
 12  runtime               3355 non-null   float64
 13  spoken_languages      3355 non-null   object 
 14  status                3355 non-null   object 
 15  tagline              

In [31]:
#Exporting and train and test dataset for the users.

train_df.to_csv("train_df.csv", index=False)
test_df.to_csv("test_df.csv", index=False)

# Expoting the unrated movies by the user 
unrated_movies.to_csv("unrated_movies.csv", index=False)